In [ ]:
# @title 75. Advanced Multimodal AI Project: Setup & Configuration (Cell 1 - FIXED)
# =========================================================
# SETUP AND IMPORTS
# =========================================================
print("Setting up environment...")
!pip install -q tensorflow transformers numpy

import tensorflow as tf
from tensorflow.keras.layers import Layer # Needed for HybridSynapseLayer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.applications import ResNet50
import numpy as np
import traceback

# --- CONFIGURATION (Global Settings) ---
IMAGE_SHAPE = (224, 224, 3)
DATA_INPUT_SIZE = 50
NUM_CLASSES = 10
FRONTAL_LOBE_UNITS = 256
TRAINING_BATCH_SIZE = 32 # Crucial for padding/static variable shape

# =========================================================
# 🛑 CRITICAL FIX: MISSING MULTIMODAL CONSTANTS
# These variables MUST be defined here for Cells 3, 5, and 7 to work.
# =========================================================
TS_STEPS = 10     # Time steps for RNN inputs (e.g., LSTMs)
TS_DIM = 1        # Feature dimension per time step
SEQ_LEN = 20      # Sequence length for text/attention inputs
SEQ_DIM = 10      # Embedding dimension per sequence step
GRAPH_DIM = 4     # Used for the flattened Graph input (4x4 = 16 features)
# =========================================================

print("Configuration complete. Constants are defined.")

Setting up environment...
Configuration complete. Constants are defined.


In [ ]:
# @title 75. Advanced Multimodal AI Project: Non-ANN Components (Cell 2)
# =========================================================
# NON-ANN COMPONENTS (Symbolic AI and SNN)
# =========================================================
import numpy as np
import tensorflow as tf
# NOTE: No MATLAB code is required.

class RuleBasedReasoner:
    """Symbolic AI for rule-based refinement."""
    def __init__(self, confidence_threshold=0.95):
        self.confidence_threshold = confidence_threshold

    def apply_rules(self, classification_probabilities, input_text):
        max_prob = np.max(classification_probabilities)

        # Rule 1: High confidence ML decision
        if max_prob >= self.confidence_threshold:
            return "ML_TRUST", np.argmax(classification_probabilities)

        # Rule 2: Symbolic Override (e.g., critical safety alert)
        if "deny" in input_text.lower() and max_prob < 0.8:
            # Assuming class '1' means 'safety denial' or 'override action'
            return "SYMBOLIC_OVERRIDE", 1

        # Default ML Decision
        return "ML_DEFAULT", np.argmax(classification_probabilities)

class LIFNeuron:
    """Spiking Neural Network (LIF) component."""
    def __init__(self, R=1.0, C=10.0, V_th=0.5):
        self.R = R
        self.C = C
        self.V_th = V_th
        self.tau = R * C
        self.V = 0.0
        self.t_ref = 0

    def step(self, I_in, dt=1.0, refractory_period=5):
        spike = 0
        if self.t_ref > 0:
            self.V = 0.0
            self.t_ref -= 1
        else:
            # LIF Model: dV/dt = (-V + R*I)/tau
            dV = ((-self.V + self.R * I_in) / self.tau) * dt
            self.V += dV
            if self.V >= self.V_th:
                spike = 1
                self.V = 0.0
                self.t_ref = refractory_period
        return spike, self.V

    def process_features(self, feature_vector):
        # Scale the feature vector sum to act as input current (I_in)
        I_in = np.sum(feature_vector)
        total_spikes = 0
        self.V = 0.0
        # Simulate 10 time steps
        for t in range(10):
            # Divide I_in by time steps to model continuous input
            spike, _ = self.step(I_in / 10.0)
            total_spikes += spike
        return total_spikes

In [ ]:
# @title 75. Advanced Multimodal AI Project: 21-Stream Model & Hybrid Layers (Cell 3 - FIXED)
# =========================================================
# CUSTOM LAYERS, FRONTAL LOBE LOGIC, and 21-STREAM ARCHITECTURE
# =========================================================

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, LSTM, Attention, Dropout, GRU, Conv2D, MaxPooling2D, Flatten, Reshape, TimeDistributed, Embedding, GlobalAveragePooling1D # Added GlobalAveragePooling1D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
import numpy as np

# --- Custom Layer Definitions (Hybrid & PFC) ---
class HybridSynapseLayer(Layer):
    """Implements synaptic plasticity and standard Keras function."""
    def __init__(self, units, plasticity_rate=0.01, decay_factor=0.8, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.plasticity_rate = plasticity_rate
        self.decay_factor = decay_factor
        self.input_activity_var = None
        self.output_activity_var = None
    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.w = self.add_weight(shape=(input_dim, self.units), initializer='random_normal', trainable=True, name='kernel')
        self.b = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True, name='bias')
        # Requires TRAINING_BATCH_SIZE (defined in Cell 1) to be defined
        self.input_activity_var = self.add_weight(name='input_activity_var', shape=(TRAINING_BATCH_SIZE, input_dim), initializer='zeros', trainable=False)
        self.output_activity_var = self.add_weight(name='output_activity_var', shape=(TRAINING_BATCH_SIZE, self.units), initializer='zeros', trainable=False)
        self.built = True
    def call(self, inputs):
        output = tf.matmul(inputs, self.w) + self.b
        output_act = tf.nn.relu(output)
        current_batch_size = tf.shape(inputs)[0]
        padding_size = TRAINING_BATCH_SIZE - current_batch_size
        padding_input = tf.zeros([padding_size, tf.shape(inputs)[1]], dtype=inputs.dtype)
        padding_output = tf.zeros([padding_size, tf.shape(output_act)[1]], dtype=output_act.dtype)
        padded_inputs = tf.concat([inputs, padding_input], axis=0)
        padded_outputs = tf.concat([output_act, padding_output], axis=0)
        # Store activity for plasticity calculation in train_step
        self.input_activity_var.assign(padded_inputs)
        self.output_activity_var.assign(padded_outputs)
        return output_act
    @tf.function
    def calculate_plasticity_change(self):
        input_act_full = self.input_activity_var
        output_act_full = self.output_activity_var
        input_act_reshaped = tf.expand_dims(input_act_full, axis=-1)
        output_act_reshaped = tf.expand_dims(output_act_full, axis=1)
        # Hebbian Term: LTP
        hebbian_term = tf.reduce_mean(tf.matmul(input_act_reshaped, output_act_reshaped), axis=0)
        ltp_change = self.plasticity_rate * hebbian_term
        # Weight Decay Term: LTD
        ltd_change = self.plasticity_rate * self.decay_factor * self.w
        delta_plasticity = ltp_change - ltd_change
        return delta_plasticity, tf.reduce_sum(tf.abs(delta_plasticity))

class ExecutiveGatingLayer(Layer):
    """PFC-like working memory layer."""
    def __init__(self, units, context_decay=0.9, context_learn_rate=0.1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.context_decay = context_decay
        self.context_learn_rate = context_learn_rate
        self.activation = tf.keras.activations.get(activation)
    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.w = self.add_weight(shape=(input_dim, self.units), initializer='random_normal', trainable=True, name='kernel')
        self.b = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True, name='bias')
        # Working memory state (trainable=False means it's updated manually/in the call)
        self.context_state = self.add_weight(shape=(self.units,), initializer='ones', trainable=False, name='working_memory_context')
        self.built = True
    def call(self, inputs):
        pre_activation = tf.matmul(inputs, self.w) + self.b
        gated_output = pre_activation * self.context_state
        output_activity = self.activation(gated_output)

        # Context Update Logic (Simulated update based on current batch activity)
        decayed_context = self.context_state * self.context_decay
        # Use the mean activity of the batch for a stable update signal
        mean_activity = tf.reduce_mean(output_activity, axis=0)
        new_signal = mean_activity * self.context_learn_rate

        self.context_state.assign(decayed_context + new_signal)
        return output_activity

class InternalPredictorLayer(Layer):
    """Internal monitoring signal layer (for internal consistency loss)."""
    def __init__(self, units, activation='linear', **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)
    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.w = self.add_weight(shape=(input_dim, self.units), initializer='random_normal', trainable=True, name='prediction_kernel')
        self.b = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True, name='prediction_bias')
        self.built = True
    def call(self, inputs):
        predicted_state = tf.matmul(inputs, self.w) + self.b
        return self.activation(predicted_state)

# --- Core Logic Function ---
def replicate_frontal_lobe_version(input_tensor, output_size):
    """The unified Frontal Lobe module."""
    predictor = InternalPredictorLayer(units=output_size, name='internal_predictor_pfc_input')(input_tensor)
    gated_output = ExecutiveGatingLayer(units=output_size, name='executive_gating_pfc_output')(input_tensor)
    return gated_output, predictor

# --- The 21-Stream Model Builder Function ---
def build_multimodal_model(image_shape, data_input_size,
                           ts_steps, ts_dim, seq_len, seq_dim,
                           graph_dim, num_classes):

    # Define placeholder variables for the new input shapes
    # NOTE: These are now properly defined in Cell 1, but we keep the parameter list

    # --- DEFINE ALL 21 INPUTS ---
    vision_input = Input(shape=image_shape, name="stream_01_image_input")
    data_input = Input(shape=(data_input_size,), name="stream_02_structured_data_input")
    ts_input_1 = Input(shape=(ts_steps, ts_dim), name="stream_03_lstm_input")
    ts_input_2 = Input(shape=(ts_steps, ts_dim), name="stream_04_gru_input")
    ts_input_3 = Input(shape=(ts_steps, ts_dim), name="stream_05_bidirectional_input")
    ts_input_4 = Input(shape=(ts_steps, ts_dim), name="stream_06_timedistributed_input")
    seq_input_1 = Input(shape=(seq_len,), name="stream_07_text_seq_input")
    seq_input_2 = Input(shape=(seq_len, seq_dim), name="stream_08_transformer_input")
    grid_input_1 = Input(shape=(32, 32, 1), name="stream_09_small_cnn_input")
    grid_input_2 = Input(shape=(64, 64, 3), name="stream_10_deep_cnn_input")
    grid_input_3 = Input(shape=(16, 16, 8), name="stream_11_conv_autoencoder_input")
    dense_input_1 = Input(shape=(100,), name="stream_12_fnn_input_100")
    dense_input_2 = Input(shape=(256,), name="stream_13_fnn_input_256")
    dense_input_3 = Input(shape=(128,), name="stream_14_vae_latent_input")
    dense_input_4 = Input(shape=(64,), name="stream_15_rbm_feature_input")
    complex_input_1 = Input(shape=(graph_dim * graph_dim,), name="stream_16_graph_input_flat")
    complex_input_2 = Input(shape=(40,), name="stream_17_attention_vector_input")
    complex_input_3 = Input(shape=(80,), name="stream_18_custom_encoder_input")
    residual_input_1 = Input(shape=(512,), name="stream_19_residual_fwd_input")
    residual_input_2 = Input(shape=(512,), name="stream_20_residual_bwd_input")
    residual_input_3 = Input(shape=(512,), name="stream_21_residual_final_input")

    all_feature_streams = []

    # --- 21 FEATURE PATHWAYS (Standardized to 512 units with HybridSynapseLayer) ---

    # 1. VISUAL (ResNet50 for Computer Vision)
    base_vision = ResNet50(weights='imagenet', include_top=False, input_tensor=vision_input, pooling='avg')
    base_vision.trainable = False
    vision_features = base_vision.output
    x_vision = HybridSynapseLayer(units=512, name="stream_01_vision_synapse_layer")(vision_features)
    x_vision = Dense(512, activation='relu', name='stream_01_vision_final_dense')(x_vision)
    all_feature_streams.append(x_vision)

    # 2. STRUCTURED DATA (FNN for Data Science)
    x_data = Dense(512, activation='relu', name='stream_02_data_dense_1')(data_input)
    x_data = HybridSynapseLayer(units=512, name="stream_02_data_synapse_layer")(x_data)
    x_data = Dense(512, activation='relu', name='stream_02_data_dense_2')(x_data)
    all_feature_streams.append(x_data)

    # 3-6. Time-Series/RNN streams
    x_ts1 = LSTM(512, return_sequences=False, name='stream_03_lstm_extractor')(ts_input_1)
    x_ts1 = HybridSynapseLayer(units=512, name="stream_03_ts1_synapse_layer")(x_ts1)
    x_ts1 = Dense(512, activation='relu', name='stream_03_ts1_final_dense')(x_ts1)
    all_feature_streams.append(x_ts1)

    x_ts2 = GRU(512, name='stream_04_gru_extractor')(ts_input_2)
    x_ts2 = HybridSynapseLayer(units=512, name="stream_04_ts2_synapse_layer")(x_ts2)
    x_ts2 = Dense(512, activation='relu', name='stream_04_ts2_final_dense')(x_ts2)
    all_feature_streams.append(x_ts2)

    x_ts3 = tf.keras.layers.Bidirectional(LSTM(256), name='stream_05_bidir_extractor')(ts_input_3)
    x_ts3 = Dense(512, activation='relu', name='stream_05_bidir_standardizer')(x_ts3)
    x_ts3 = HybridSynapseLayer(units=512, name="stream_05_ts3_synapse_layer")(x_ts3)
    x_ts3 = Dense(512, activation='relu', name='stream_05_ts3_final_dense')(x_ts3)
    all_feature_streams.append(x_ts3)

    x_ts4 = TimeDistributed(Dense(32), name='stream_06_td_processor')(ts_input_4)
    x_ts4 = Flatten(name='stream_06_td_flatten')(x_ts4)
    x_ts4 = Dense(512, activation='relu', name='stream_06_td_standardizer')(x_ts4)
    x_ts4 = HybridSynapseLayer(units=512, name="stream_06_ts4_synapse_layer")(x_ts4)
    x_ts4 = Dense(512, activation='relu', name='stream_06_ts4_final_dense')(x_ts4)
    all_feature_streams.append(x_ts4)

    # 7-8. NLP/Sequential streams
    x_seq1 = Embedding(input_dim=1000, output_dim=64, name='stream_07_embedding')(seq_input_1)
    x_seq1 = GRU(512, name='stream_07_embedding_gru')(x_seq1)
    x_seq1 = HybridSynapseLayer(units=512, name="stream_07_seq1_synapse_layer")(x_seq1)
    x_seq1 = Dense(512, activation='relu', name='stream_07_seq1_final_dense')(x_seq1)
    all_feature_streams.append(x_seq1)

    attention_out = Attention(use_scale=True, name='stream_08_attention')([seq_input_2, seq_input_2])
    # 🛑 CRITICAL FIX APPLIED HERE: Replaced tf.reduce_mean with GlobalAveragePooling1D
    x_seq2 = GlobalAveragePooling1D(name='stream_08_attention_pool')(attention_out)
    x_seq2 = Dense(512, activation='relu', name='stream_08_transformer_standardizer')(x_seq2)
    x_seq2 = HybridSynapseLayer(units=512, name="stream_08_seq2_synapse_layer")(x_seq2)
    x_seq2 = Dense(512, activation='relu', name='stream_08_seq2_final_dense')(x_seq2)
    all_feature_streams.append(x_seq2)

    # 9-11. CNN/Grid streams
    x_grid1 = Conv2D(32, (3, 3), activation='relu', name='stream_09_conv1')(grid_input_1)
    x_grid1 = MaxPooling2D((2, 2), name='stream_09_pool1')(x_grid1)
    x_grid1 = Flatten(name='stream_09_flatten')(x_grid1)
    x_grid1 = Dense(512, activation='relu', name='stream_09_cnn_standardizer')(x_grid1)
    x_grid1 = HybridSynapseLayer(units=512, name="stream_09_grid1_synapse_layer")(x_grid1)
    x_grid1 = Dense(512, activation='relu', name='stream_09_grid1_final_dense')(x_grid1)
    all_feature_streams.append(x_grid1)

    x_grid2 = Conv2D(64, (5, 5), activation='relu', name='stream_10_conv_deep')(grid_input_2)
    x_grid2 = Flatten(name='stream_10_flatten_deep')(x_grid2)
    x_grid2 = Dense(512, activation='relu', name='stream_10_deep_cnn_standardizer')(x_grid2)
    x_grid2 = HybridSynapseLayer(units=512, name="stream_10_grid2_synapse_layer")(x_grid2)
    x_grid2 = Dense(512, activation='relu', name='stream_10_grid2_final_dense')(x_grid2)
    all_feature_streams.append(x_grid2)

    # Encoder output (from a Generative AI model like an Autoencoder)
    x_grid3 = Flatten(name='stream_11_flatten')(grid_input_3)
    x_grid3 = Dense(512, activation='relu', name='stream_11_autoencoder_input_proj')(x_grid3)
    x_grid3 = HybridSynapseLayer(units=512, name="stream_11_grid3_synapse_layer")(x_grid3)
    x_grid3 = Dense(512, activation='relu', name='stream_11_grid3_final_dense')(x_grid3)
    all_feature_streams.append(x_grid3)

    # 12-15. Dense/Vector streams (VAE/RBM for Generative AI/Deep Learning)
    x_dense1 = Dense(1024, activation='relu', name='stream_12_dense_1')(dense_input_1)
    x_dense1 = Dense(512, activation='relu', name='stream_12_fnn_standardizer')(x_dense1)
    x_dense1 = HybridSynapseLayer(units=512, name="stream_12_dense1_synapse_layer")(x_dense1)
    x_dense1 = Dense(512, activation='relu', name='stream_12_dense1_final_dense')(x_dense1)
    all_feature_streams.append(x_dense1)

    x_dense2 = Dense(512, activation='relu', name='stream_13_dense_1')(dense_input_2)
    x_dense2 = HybridSynapseLayer(units=512, name="stream_13_dense2_synapse_layer")(x_dense2)
    x_dense2 = Dense(512, activation='relu', name='stream_13_dense2_final_dense')(x_dense2)
    all_feature_streams.append(x_dense2)

    x_dense3 = Dense(512, activation='relu', name='stream_14_vae_proj')(dense_input_3)
    x_dense3 = HybridSynapseLayer(units=512, name="stream_14_dense3_synapse_layer")(x_dense3)
    x_dense3 = Dense(512, activation='relu', name='stream_14_dense3_final_dense')(x_dense3)
    all_feature_streams.append(x_dense3)

    x_dense4 = Dense(512, activation='relu', name='stream_15_rbm_proj')(dense_input_4)
    x_dense4 = HybridSynapseLayer(units=512, name="stream_15_dense4_synapse_layer")(x_dense4)
    x_dense4 = Dense(512, activation='relu', name='stream_15_dense4_final_dense')(x_dense4)
    all_feature_streams.append(x_dense4)

    # 16-18. Complex streams (Graph/Attention)
    x_complex1 = Dense(1024, activation='relu', name='stream_16_graph_dense_1')(complex_input_1)
    x_complex1 = Dense(512, activation='relu', name='stream_16_graph_standardizer')(x_complex1)
    x_complex1 = HybridSynapseLayer(units=512, name="stream_16_complex1_synapse_layer")(x_complex1)
    x_complex1 = Dense(512, activation='relu', name='stream_16_complex1_final_dense')(x_complex1)
    all_feature_streams.append(x_complex1)

    x_complex2 = Dense(512, activation='relu', name='stream_17_attention_proj')(complex_input_2)
    x_complex2 = HybridSynapseLayer(units=512, name="stream_17_complex2_synapse_layer")(x_complex2)
    x_complex2 = Dense(512, activation='relu', name='stream_17_complex2_final_dense')(x_complex2)
    all_feature_streams.append(x_complex2)

    x_complex3 = Dense(512, activation='relu', name='stream_18_custom_encoder_proj')(complex_input_3)
    x_complex3 = HybridSynapseLayer(units=512, name="stream_18_complex3_synapse_layer")(x_complex3)
    x_complex3 = Dense(512, activation='relu', name='stream_18_complex3_final_dense')(x_complex3)
    all_feature_streams.append(x_complex3)

    # 19-21. Residual Fused streams
    x_res1 = HybridSynapseLayer(units=512, name="stream_19_res1_synapse_layer")(residual_input_1)
    # Using tf.keras.layers.Add is correct and Keras-compatible
    x_res1 = tf.keras.layers.Add(name='stream_19_residual_add_forward')([x_res1, residual_input_1])
    x_res1 = Dense(512, activation='relu', name='stream_19_res1_final_dense')(x_res1)
    all_feature_streams.append(x_res1)

    x_res2 = HybridSynapseLayer(units=512, name="stream_20_res2_synapse_layer")(residual_input_2)
    x_res2 = tf.keras.layers.Add(name='stream_20_residual_add_backward')([x_res2, residual_input_2])
    x_res2 = Dense(512, activation='relu', name='stream_20_res2_final_dense')(x_res2)
    all_feature_streams.append(x_res2)

    x_res3 = HybridSynapseLayer(units=512, name="stream_21_res3_synapse_layer")(residual_input_3)
    x_res3 = tf.keras.layers.Add(name='stream_21_residual_add_final')([x_res3, residual_input_3])
    x_res3 = Dense(512, activation='relu', name='stream_21_res3_final_dense')(x_res3)
    all_feature_streams.append(x_res3)

    # --- FINAL FUSION AND HYBRID CORE ---
    combined_features = Concatenate(name="multimodal_fusion")(all_feature_streams)

    core_features = Dense(512, activation='relu', name='core_dense_pre_custom')(combined_features)
    # Apply Hybrid Synapse logic to the central feature fusion block
    core_synapse_output = HybridSynapseLayer(units=512, name="core_synapse_layer")(core_features)

    # Apply Frontal Lobe Logic
    gated_output, predictor_output = replicate_frontal_lobe_version(core_synapse_output, FRONTAL_LOBE_UNITS)

    final_output = Dense(num_classes, activation='softmax', name="main_classification_output")(gated_output)

    # Model definition with ALL 21 inputs:
    model = Model(inputs=[vision_input, data_input, ts_input_1, ts_input_2, ts_input_3, ts_input_4,
                          seq_input_1, seq_input_2, grid_input_1, grid_input_2, grid_input_3,
                          dense_input_1, dense_input_2, dense_input_3, dense_input_4,
                          complex_input_1, complex_input_2, complex_input_3,
                          residual_input_1, residual_input_2, residual_input_3],
                  outputs=[final_output, predictor_output])
    return model

In [ ]:
# @title 75. Advanced Multimodal AI Project: Custom Training & Simultaneous Update

# =========================================================
# CUSTOM TRAINING LOOP (Simultaneous Logic with ID() Fix)
# =========================================================

class HybridMultiModalModel(tf.keras.Model):
    """Custom model for managing all losses and the simultaneous hybrid weight update."""
    def __init__(self, multimodal_core_model):
        super().__init__()
        self.multimodal_core = multimodal_core_model
        self.hybrid_layers = [l for l in multimodal_core_model.layers if isinstance(l, HybridSynapseLayer)]
        self.internal_loss_fn = tf.keras.losses.MeanSquaredError(name='internal_loss')

    def compile(self, optimizer, loss, metrics=None):
        # NOTE: Your classification loss is passed here, but the internal loss is calculated manually.
        super().compile(optimizer=optimizer, loss=loss, metrics=metrics)

    @tf.function
    def train_step(self, data):
        (x_data, y_targets) = data
        x_inputs = x_data
        y_true_classification = y_targets[0]
        y_true_internal_feature = y_targets[1]

        plasticity_deltas = {}
        total_plasticity_mag = tf.constant(0.0)

        with tf.GradientTape() as tape:
            y_pred_classification, y_pred_internal = self.multimodal_core(x_inputs, training=True)

            # 🛑 FIX 1: Use compute_loss() instead of compiled_loss()
            # This requires passing the input data 'x_inputs' as the first argument.
            # Old: classification_loss = self.compiled_loss(y_true_classification, y_pred_classification, regularization_losses=self.losses)
            classification_loss = self.compute_loss(
                x_inputs, y_true_classification, y_pred_classification, regularization_losses=self.losses
            )

            internal_consistency_loss = self.internal_loss_fn(y_true_internal_feature, y_pred_internal)
            total_loss = classification_loss + (0.1 * internal_consistency_loss)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(total_loss, trainable_vars)

        # 1. Collect Non-Gradient Plasticity Terms (ΔW_Plasticity) - **NO CHANGE HERE**
        for layer in self.hybrid_layers:
            delta_p, mag_p = layer.calculate_plasticity_change()
            plasticity_deltas[id(layer.w)] = delta_p
            total_plasticity_mag += mag_p

        # 2. Combine Plasticity with Backprop Gradient - **NO CHANGE HERE**
        combined_gradients = []
        for grad, var in zip(gradients, trainable_vars):
            if id(var) in plasticity_deltas:
                # Enforce W_new = W_old + ΔW_Backprop + ΔW_Plasticity
                # Note: The backprop term 'grad' is actually -ΔW_Backprop, so we subtract
                # the plasticity delta (ΔW_Plasticity) to effectively add it to the weight update.
                grad = grad - plasticity_deltas[id(var)]

            combined_gradients.append(grad)

        # 3. Apply the Combined Gradient - **NO CHANGE HERE**
        self.optimizer.apply_gradients(zip(combined_gradients, trainable_vars))

        # 🛑 FIX 2: Use compute_metrics() instead of compiled_metrics.update_state()
        # This requires passing the input data 'x_inputs' as the first argument.
        # Old: self.compiled_metrics.update_state(y_true_classification, y_pred_classification)
        self.compute_metrics(x_inputs, y_true_classification, y_pred_classification)


        # The return value is correctly structured to report all metrics
        return {m.name: m.result() for m in self.metrics} | \
               {"classification_loss": classification_loss,
                "internal_loss": internal_consistency_loss,
                "plasticity_update_mag": total_plasticity_mag}

In [ ]:
# @title 75. Advanced Multimodal AI Project: Execution (Cell 5)
# =========================================================
# MAIN EXECUTION BLOCK (FIXED for 21 Inputs)
# =========================================================
import numpy as np
import tensorflow as tf
import traceback
from tensorflow.keras.applications import ResNet50 # Re-import just in case

# --- System Setup ---
def setup_system():
    # Placeholder values for build_multimodal_model (actual values are defined in Cell 1)
    # The arguments here are just used for the function signature but are not used internally in the fixed build_multimodal_model
    multimodal_core = build_multimodal_model(IMAGE_SHAPE, DATA_INPUT_SIZE,
                                             TS_STEPS, TS_DIM, SEQ_LEN, SEQ_DIM,
                                             GRAPH_DIM, NUM_CLASSES)
    hybrid_model = HybridMultiModalModel(multimodal_core)
    hybrid_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[tf.keras.metrics.CategoricalAccuracy()]
    )
    # RuleBasedReasoner and LIFNeuron are already defined in Cell 2
    rule_reasoner = RuleBasedReasoner()
    snn_neuron = LIFNeuron()
    return hybrid_model, multimodal_core, rule_reasoner, snn_neuron

# --- Execution ---
print("\n--- INITIALIZING PROJECT ---")
hybrid_model, multimodal_core, rule_reasoner, snn_neuron = setup_system()

# Dummy Data Setup for Training (Fixed for 21 Streams)
N_SAMPLES = 4096
print(f"Creating dummy data for {N_SAMPLES} samples and 21 streams...")

# List of 21 Dummy Inputs
X_inputs = [
    # 1. Vision
    np.random.rand(N_SAMPLES, *IMAGE_SHAPE).astype(np.float32),      # stream_01_image_input
    # 2. Structured Data
    np.random.rand(N_SAMPLES, DATA_INPUT_SIZE).astype(np.float32),   # stream_02_structured_data_input
    # 3-6. Time-Series / RNN
    np.random.rand(N_SAMPLES, TS_STEPS, TS_DIM).astype(np.float32),  # stream_03_lstm_input
    np.random.rand(N_SAMPLES, TS_STEPS, TS_DIM).astype(np.float32),  # stream_04_gru_input
    np.random.rand(N_SAMPLES, TS_STEPS, TS_DIM).astype(np.float32),  # stream_05_bidirectional_input
    np.random.rand(N_SAMPLES, TS_STEPS, TS_DIM).astype(np.float32),  # stream_06_timedistributed_input
    # 7-8. NLP / Sequential
    np.random.randint(0, 1000, size=(N_SAMPLES, SEQ_LEN)),           # stream_07_text_seq_input (Integer indices)
    np.random.rand(N_SAMPLES, SEQ_LEN, SEQ_DIM).astype(np.float32),  # stream_08_transformer_input
    # 9-11. Grid / CNN
    np.random.rand(N_SAMPLES, 32, 32, 1).astype(np.float32),         # stream_09_small_cnn_input
    np.random.rand(N_SAMPLES, 64, 64, 3).astype(np.float32),         # stream_10_deep_cnn_input
    np.random.rand(N_SAMPLES, 16, 16, 8).astype(np.float32),         # stream_11_conv_autoencoder_input
    # 12-15. Dense / Vector
    np.random.rand(N_SAMPLES, 100).astype(np.float32),               # stream_12_fnn_input_100
    np.random.rand(N_SAMPLES, 256).astype(np.float32),              # stream_13_fnn_input_256
    np.random.rand(N_SAMPLES, 128).astype(np.float32),              # stream_14_vae_latent_input
    np.random.rand(N_SAMPLES, 64).astype(np.float32),                # stream_15_rbm_feature_input
    # 16-18. Complex / Graph
    np.random.rand(N_SAMPLES, GRAPH_DIM * GRAPH_DIM).astype(np.float32), # stream_16_graph_input_flat
    np.random.rand(N_SAMPLES, 40).astype(np.float32),                # stream_17_attention_vector_input
    np.random.rand(N_SAMPLES, 80).astype(np.float32),                # stream_18_custom_encoder_input
    # 19-21. Residual Fused
    np.random.rand(N_SAMPLES, 512).astype(np.float32),               # stream_19_residual_fwd_input
    np.random.rand(N_SAMPLES, 512).astype(np.float32),               # stream_20_residual_bwd_input
    np.random.rand(N_SAMPLES, 512).astype(np.float32),               # stream_21_residual_final_input
]

# Targets (Labels for the two model outputs)
Y_classification = tf.one_hot(np.random.randint(0, NUM_CLASSES, N_SAMPLES), depth=NUM_CLASSES)
Y_internal_feature = np.random.rand(N_SAMPLES, FRONTAL_LOBE_UNITS).astype(np.float32)
Y_targets = [Y_classification, Y_internal_feature]

# Training Execution
print(f"\n--- Starting FULL Hybrid MultiModal Training ({N_SAMPLES} Samples, 25 Epochs) ---")
print("✅ Simultaneous Hybrid Update Logic is ENFORCED and running on a larger dataset with 21 inputs.")
try:
    history = hybrid_model.fit(X_inputs, Y_targets, epochs=25, batch_size=TRAINING_BATCH_SIZE, verbose=1)
    print("\nTraining Complete. Hybrid logic is fully functional and dynamically stable.")
except Exception as e:
    print("\n\n----------------------------------------------------")
    print("🚨 ERROR CAUGHT DURING MODEL TRAINING 🚨")
    print("----------------------------------------------------")
    print("Error Type:", type(e).__name__)
    print("Error Message:", e)
    print("\n--- Full Traceback ---")
    print(traceback.format_exc())
    print("----------------------------------------------------")


--- INITIALIZING PROJECT ---
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Creating dummy data for 4096 samples and 21 streams...


In [ ]:
# @title 75. Advanced Multimodal AI Project: Save Final Weights (CORRECTED)

print("\n--- Saving Final Learned Hybrid Weights ---")

# Ensure the hybrid_model object exists and is defined from Cell 5 execution
if 'hybrid_model' in locals():
    try:
        # FIX: Changed the filename to end with .weights.h5 as required by Keras
        hybrid_model.multimodal_core.save_weights('final_hybrid_core_weights.weights.h5')

        print(f"✅ Successfully saved hybrid core weights to 'final_hybrid_core_weights.weights.h5'.")

        # --- Optional: Save Rule-Based State (if applicable) ---
        final_context_state = hybrid_model.multimodal_core.get_layer('executive_gating_pfc_output').context_state.numpy()
        np.save('final_executive_context.npy', final_context_state)
        print(f"✅ Saved final Executive Gating Context State to 'final_executive_context.npy'.")

    except Exception as e:
        print(f"❌ ERROR saving weights: {e}")
        print("Make sure Cell 5 finished training successfully.")
else:
    print("❌ ERROR: 'hybrid_model' not found. Please run Cell 5 first to define and train the model.")

In [ ]:
# @title 75. Advanced Multimodal AI Project: Full Inference & Hybrid Reasoning (Cell 7)
# =========================================================
# FULL INFERENCE BLOCK (FIXED for 21 Inputs & Hybrid Logic)
# =========================================================
import numpy as np
import tensorflow as tf

# NOTE: This relies on 'hybrid_model', 'rule_reasoner', and 'snn_neuron' from Cell 5

print("\n--- Starting Full Hybrid System Inference Test ---")

# --- 1. Prepare Single Sample Data for Inference (Fixed for 21 Streams) ---
def prepare_single_sample_inputs():
    """Generates a single, representative sample across all 21 streams."""

    # Use the same shapes defined in Cell 1
    ts_steps, ts_dim, seq_len, seq_dim, graph_dim = TS_STEPS, TS_DIM, SEQ_LEN, SEQ_DIM, GRAPH_DIM

    test_inputs = [
        # 1. Vision
        np.random.rand(1, *IMAGE_SHAPE).astype(np.float32),      # stream_01_image_input
        # 2. Structured Data
        np.random.rand(1, DATA_INPUT_SIZE).astype(np.float32),   # stream_02_structured_data_input
        # 3-6. Time-Series / RNN
        np.random.rand(1, ts_steps, ts_dim).astype(np.float32),  # stream_03_lstm_input
        np.random.rand(1, ts_steps, ts_dim).astype(np.float32),  # stream_04_gru_input
        np.random.rand(1, ts_steps, ts_dim).astype(np.float32),  # stream_05_bidirectional_input
        np.random.rand(1, ts_steps, ts_dim).astype(np.float32),  # stream_06_timedistributed_input
        # 7-8. NLP / Sequential
        np.random.randint(0, 1000, size=(1, seq_len)),           # stream_07_text_seq_input (Integer indices)
        np.random.rand(1, seq_len, seq_dim).astype(np.float32),  # stream_08_transformer_input
        # 9-11. Grid / CNN
        np.random.rand(1, 32, 32, 1).astype(np.float32),         # stream_09_small_cnn_input
        np.random.rand(1, 64, 64, 3).astype(np.float32),         # stream_10_deep_cnn_input
        np.random.rand(1, 16, 16, 8).astype(np.float32),         # stream_11_conv_autoencoder_input
        # 12-15. Dense / Vector
        np.random.rand(1, 100).astype(np.float32),               # stream_12_fnn_input_100
        np.random.rand(1, 256).astype(np.float32),              # stream_13_fnn_input_256
        np.random.rand(1, 128).astype(np.float32),              # stream_14_vae_latent_input
        np.random.rand(1, 64).astype(np.float32),                # stream_15_rbm_feature_input
        # 16-18. Complex / Graph
        np.random.rand(1, graph_dim * graph_dim).astype(np.float32), # stream_16_graph_input_flat
        np.random.rand(1, 40).astype(np.float32),                # stream_17_attention_vector_input
        np.random.rand(1, 80).astype(np.float32),                # stream_18_custom_encoder_input
        # 19-21. Residual Fused
        np.random.rand(1, 512).astype(np.float32),               # stream_19_residual_fwd_input
        np.random.rand(1, 512).astype(np.float32),               # stream_20_residual_bwd_input
        np.random.rand(1, 512).astype(np.float32),               # stream_21_residual_final_input
    ]

    # Create a representative text input for the symbolic reasoner (Critical example)
    representative_text_input = "The system is performing optimally and no critical alerts."

    # Generate a dummy feature vector for the SNN (use a slice of the main feature set)
    snn_feature_vector = np.random.rand(30).astype(np.float32)

    return test_inputs, representative_text_input, snn_feature_vector

# --- 2. Execute Full Hybrid Inference Pipeline ---
def run_hybrid_inference(hybrid_model, rule_reasoner, snn_neuron, test_inputs, text_input, snn_features):

    # 2.1. Neural Network Prediction
    # The core model returns the classification probabilities and the internal PFC features
    nn_output, internal_prediction = hybrid_model.multimodal_core.predict(test_inputs, verbose=0)

    nn_prob = nn_output[0]
    nn_pred_class = np.argmax(nn_prob)

    print(f"\n[ANN/PFC Core] Raw NN Prediction: Class {nn_pred_class} (Confidence: {np.max(nn_prob):.4f})")

    # 2.2. Spiking Neural Network (SNN) Analysis
    # The SNN evaluates a small feature set for a secondary opinion (e.g., vigilance/anomaly score)
    snn_spikes = snn_neuron.process_features(snn_features)
    snn_threshold = 2 # Example threshold
    snn_signal = "ALERT" if snn_spikes > snn_threshold else "NORMAL"

    print(f"[SNN/Vigilance] Total Spikes: {snn_spikes}. SNN Signal: {snn_signal}.")

    # 2.3. Symbolic AI (Rule-Based Refinement)
    # The RuleReasoner takes the NN output and the Symbolic text input (e.g., a command or log message)
    reasoning_type, final_class = rule_reasoner.apply_rules(nn_prob, text_input)

    print(f"\n[Symbolic/Executive] Reasoning Type: {reasoning_type}")
    print(f"[Symbolic/Executive] Final Decision Class: {final_class}")

    # Check for SNN influence (simple rule: if SNN is ALERT, bump the class up by 1)
    if snn_signal == "ALERT" and reasoning_type != "SYMBOLIC_OVERRIDE":
        # The SNN acts as an executive modifier if rules haven't already overridden
        final_class = np.minimum(final_class + 1, NUM_CLASSES - 1)
        reasoning_type += " + SNN_MODIFIER"
        print(f"[SNN Effect] SNN ALERT triggered class modification. New Final Class: {final_class}")

    return final_class, reasoning_type

# --- Run the full test ---
if 'hybrid_model' in locals() and 'rule_reasoner' in locals() and 'snn_neuron' in locals():

    # Create the data for the test run
    test_inputs, text_input, snn_features = prepare_single_sample_inputs()

    # Run the hybrid pipeline
    final_decision_class, final_reasoning = run_hybrid_inference(
        hybrid_model, rule_reasoner, snn_neuron, test_inputs, text_input, snn_features
    )

    print("\n----------------------------------------------------")
    print("✅ FULL HYBRID SYSTEM DECISION COMPLETE ✅")
    print(f"Final Class Decision: {final_decision_class}")
    print(f"Reasoning Path: {final_reasoning}")
    print("----------------------------------------------------")

else:
    print("❌ ERROR: Required variables ('hybrid_model', 'rule_reasoner', 'snn_neuron') not found.")
    print("Please ensure Cell 5 (Execution) and Cell 2 (Non-ANN) ran successfully.")